In [2]:
import pickle

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tqdm.notebook import tqdm

from simulator.simulator import Sim
from strategies.baselines import BestPosStrategy, StoikovStrategy
from utils.get_info import get_pnl, get_volumes
from utils.load_data import load_md_from_file

PATH_TO_FILE = 'md/btcusdt:Binance:LinearPerpetual/'
# PATH_TO_FILE = 'md_new/ethusdt/'
NROWS = 10_000_000

In [4]:
md = load_md_from_file(path=PATH_TO_FILE, nrows=NROWS, btc=True)

latency = pd.Timedelta(10, 'ms').delta
md_latency = pd.Timedelta(10, 'ms').delta

# BestPosStrategy

In [5]:
sim = Sim(md[1_080_000:], latency, md_latency)
# sim = Sim(md[230_000:], latency, md_latency)

delay = pd.Timedelta(0.1, 's').delta
hold_time = pd.Timedelta(10, 's').delta
strategy = BestPosStrategy(delay, hold_time, trade_size=0.001)

In [8]:
%%time
trades_list_bp, md_list, updates_list, all_orders = strategy.run(sim)

CPU times: user 2min 39s, sys: 4.72 s, total: 2min 43s
Wall time: 2min 49s


In [10]:
%%time
df_bp = get_pnl(updates_list, post_only=True)
df_bp['receive_ts'] = pd.to_datetime(df_bp['receive_ts'])

CPU times: user 10min 3s, sys: 13 s, total: 10min 16s
Wall time: 10min 32s


In [11]:
%%time
df_bp_fee = get_pnl(updates_list, post_only=True, maker_fee=-0.00004, taker_fee=0.0004)
df_bp_fee['receive_ts'] = pd.to_datetime(df_bp_fee['receive_ts'])

CPU times: user 9min 50s, sys: 9.49 s, total: 9min 59s
Wall time: 10min 8s


In [12]:
ask_made, bid_made, ask_take, bid_take = get_volumes(trades_list_bp)
ask_made, bid_made, ask_take, bid_take

(613.0979999924141, 614.4379999923824, 19.2970000000006, 19.601000000000973)

In [40]:
df_st_fee.to_csv('df_st_fee.csv')

In [ ]:
fig = make_subplots(rows=3, cols=1, subplot_titles=("Price", "PnL", "Inventory Size"))

fig.add_trace(go.Scatter(x=df_bp.iloc[::100, :]['receive_ts'], y=df_bp.iloc[::100, :]['mid_price'],
                         name='Price'), row=1, col=1)

fig.add_trace(go.Scatter(x=df_bp.iloc[::100, :]['receive_ts'], y=df_bp.iloc[::100, :]['total'],
                         name='PnL without fees'), row=2, col=1)
fig.add_trace(go.Scatter(x=df_bp_fee.iloc[::100, :]['receive_ts'], y=df_bp_fee.iloc[::100, :]['total'],
                         name='PnL including fees'), row=2, col=1)

fig.add_trace(go.Scatter(x=df_bp.iloc[::100, :]['receive_ts'], y=df_bp.iloc[::100, :]['BTC'],
                         name='Inventory Size'), row=3, col=1)

fig.update_yaxes(title_text="USDT", row=1, col=1)
fig.update_yaxes(title_text="USDT", row=2, col=1)
fig.update_yaxes(title_text="BTC", row=3, col=1)

fig.update_layout(title_text="Naive Strategy: maker fee = -0.004%", height=700)

fig.write_html('docs/NaiveStrategy_btc.html')
# fig.write_image('../images/results/NaiveStrategy.jpeg')
fig.show()

# Stoikov Avellaneda Strategy

### k

In [35]:
dfs_k = []

ks = [10, 12, 15]
# ks = [1, 10, 20]

for k in tqdm(ks):
    sim = Sim(md[:230_000], latency, md_latency)

    delay = pd.Timedelta(0.1, 's').delta
    hold_time = pd.Timedelta(10, 's').delta
    terminal_date = pd.to_datetime('2022-06-24')

    strategy = StoikovStrategy(delay=delay, hold_time=hold_time, trade_size=0.01, risk_aversion=0.5, k=k,
                               post_only=True)

    trades_list_st, md_list, updates_list, all_orders = strategy.run(sim)

    df = get_pnl(updates_list, post_only=True, maker_fee=-0.00004, taker_fee=0.0004)
    df['receive_ts'] = pd.to_datetime(df['receive_ts'])
    df.set_index('receive_ts', inplace=True)

    dfs_k.append(df)

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
fig = make_subplots(rows=2, cols=1)

for i, df_ in enumerate(dfs_k):
    df__ = df_[['total', 'BTC']].resample('1s').last().reset_index()
    fig.add_trace(go.Scatter(x=df__['receive_ts'], y=df__['total'], name=f'k: {ks[i]}_USD'),
                  row=1, col=1)
    fig.add_trace(go.Scatter(x=df__['receive_ts'], y=df__['BTC'], name=f'k: {ks[i]}_ETH'),
                  row=2, col=1)

fig.update_layout(
    title_text="PnL Depending on k"
)

fig.update_xaxes(title_text="Time")
fig.update_yaxes(title_text="USDT", row=1, col=1)
fig.update_yaxes(title_text="ETH", row=2, col=1)
fig.show()
# fig.write_html('../docs/StoikovStrategy_risk_ref_430.html')

## Final Stoikov Model

In [15]:
sim = Sim(md[1_080_002:], latency, md_latency)
# sim = Sim(md[230_000:], latency, md_latency)

In [16]:
delay = pd.Timedelta(0.1, 's').delta
hold_time = pd.Timedelta(10, 's').delta

strategy = StoikovStrategy(delay=delay, hold_time=hold_time, trade_size=0.001, risk_aversion=0.5, k=1.5,
                           post_only=True)

In [18]:
%%time
trades_list_st, md_list, updates_list, all_orders = strategy.run(sim)

CPU times: user 2min 41s, sys: 6.73 s, total: 2min 48s
Wall time: 2min 57s


In [19]:
%%time
df_st = get_pnl(updates_list, post_only=True, maker_fee=0, taker_fee=0)
df_st['receive_ts'] = pd.to_datetime(df_st['receive_ts'])

CPU times: user 9min 14s, sys: 9.38 s, total: 9min 23s
Wall time: 9min 33s


In [20]:
%%time
df_st_fee = get_pnl(updates_list, post_only=True, maker_fee=-0.00004, taker_fee=0.0004)
df_st_fee['receive_ts'] = pd.to_datetime(df_st_fee['receive_ts'])

CPU times: user 9min 13s, sys: 7.56 s, total: 9min 21s
Wall time: 9min 25s


In [21]:
ask_made, bid_made, ask_take, bid_take = get_volumes(trades_list_st)
ask_made, bid_made, ask_take, bid_take

(31.581000000015614, 31.599000000015636, 4.308999999999774, 4.157999999999723)

In [ ]:
fig = make_subplots(rows=3, cols=1, subplot_titles=("Price", "PnL", "Inventory Size"))

fig.add_trace(go.Scatter(x=df_st.iloc[::100, :]['receive_ts'], y=df_st.iloc[::100, :]['mid_price'],
                         name='Price'), row=1, col=1)

fig.add_trace(go.Scatter(x=df_st.iloc[::100, :]['receive_ts'], y=df_st.iloc[::100, :]['total'],
                         name='PnL without fees'), row=2, col=1)
fig.add_trace(go.Scatter(x=df_st_fee.iloc[::100, :]['receive_ts'], y=df_st_fee.iloc[::100, :]['total'],
                         name='PnL including fees'), row=2, col=1)

fig.add_trace(go.Scatter(x=df_st.iloc[::100, :]['receive_ts'], y=df_st.iloc[::100, :]['BTC'],
                         name='Inventory Size'), row=3, col=1)

fig.update_yaxes(title_text="USDT", row=1, col=1)
fig.update_yaxes(title_text="USDT", row=2, col=1)
fig.update_yaxes(title_text="BTC", row=3, col=1)

fig.update_layout(title_text="Stoikov Strategy: maker fee = -0.004%", height=700)

# fig.write_html('docs/StoikovStrategy_btc.html')
# fig.write_image('../images/results/StoikovStrategy_eth_al_post.jpeg')
fig.show()

# RL Strategy

In [24]:
import torch

from strategies.rl import A2CNetwork, Policy, RLStrategy, ComputeValueTargets

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cpu')

In [25]:
with open('data/features_dict.pickle', 'rb') as f:
    ess_dict = pickle.load(f)

ess_df = pd.DataFrame.from_dict(ess_dict, orient='index').reset_index().rename(columns={'index': 'receive_ts'})

with open('data/means.npy', 'rb') as f:
    means = np.load(f)

with open('data/stds.npy', 'rb') as f:
    stds = np.load(f)

del(ess_dict)

In [26]:
model = A2CNetwork(n_actions=10, DEVICE=DEVICE).to(DEVICE)
model.load_state_dict(torch.load("models/A2C_1000step_ETH_25_20_post_delays_10act_rew.pth"))
model.eval()

policy = Policy(model)

delay = pd.Timedelta(0.1, 's').delta
hold_time = pd.Timedelta(10, 's').delta
strategy = RLStrategy(policy, ess_df, 1.0,
                      means, stds, delay, hold_time, [ComputeValueTargets(policy)],
                      trade_size=0.01, post_only=True, taker_fee=0.0004, maker_fee=-0.00004)

In [ ]:
% % time
strategy.reset()
sim = Sim(md[1_080_002:],
          execution_latency=pd.Timedelta(10, 'ms').delta,
          md_latency=pd.Timedelta(10, 'ms').delta)
with torch.no_grad():
    trades_list, md_list, updates_list, actions_history, trajectory = strategy.run(sim, mode='test')

In [2]:
# with open('../data/rl_model/trades_list_ETH.pickle', 'wb') as f:
#     pickle.dump(trades_list, f)

# with open('../data/rl_model/md_list_ETH.pickle', 'wb') as f:
#     pickle.dump(md_list, f)

# with open('../data/rl_model/updates_list_ETH.pickle', 'wb') as f:
#     pickle.dump(updates_list, f)

# with open('../data/rl_model/actions_history_ETH.pickle', 'wb') as f:
#     pickle.dump(actions_history, f)

# with open('data/rl_model/trades_list_BTC.pickle', 'rb') as f:
#     trades_list = pickle.load(f)

# with open('data/rl_model/md_list_BTC.pickle', 'rb') as f:
#     md_list = pickle.load(f)

# with open('data/rl_model/updates_list_BTC.pickle', 'rb') as f:
#     updates_list = pickle.load(f)

# with open('data/rl_model/actions_history_BTC.pickle', 'rb') as f:
#     actions_history = pickle.load(f)

In [8]:
%%time

df_rl = get_pnl(updates_list, post_only=True, maker_fee=0, taker_fee=0)
df_rl['receive_ts'] = pd.to_datetime(df_rl['receive_ts'])

CPU times: user 8min 52s, sys: 1min 24s, total: 10min 16s
Wall time: 11min 4s


In [9]:
%%time

df_rl_fee = get_pnl(updates_list, post_only=True, maker_fee=-0.00004, taker_fee=0.0004)
df_rl_fee['receive_ts'] = pd.to_datetime(df_rl_fee['receive_ts'])

CPU times: user 8min 52s, sys: 1min 15s, total: 10min 8s
Wall time: 10min 50s


In [12]:
ask_made, bid_made, ask_take, bid_take = get_volumes(trades_list)
ask_made, bid_made, ask_take, bid_take

(445.9899999963656, 445.3089999963817, 18.790999999999983, 19.129000000000396)

In [ ]:
fig = make_subplots(rows=3, cols=1, subplot_titles=("Price", "PnL", "Inventory Size"))

fig.add_trace(go.Scatter(x=df_rl.iloc[::100, :]['receive_ts'], y=df_rl.iloc[::100, :]['mid_price'],
                         name='Price'), row=1, col=1)

fig.add_trace(go.Scatter(x=df_rl.iloc[::100, :]['receive_ts'], y=df_rl.iloc[::100, :]['total'],
                         name='PnL without fees'), row=2, col=1)
fig.add_trace(go.Scatter(x=df_rl_fee.iloc[::100, :]['receive_ts'], y=df_rl_fee.iloc[::100, :]['total'],
                         name='PnL including fees'), row=2, col=1)

fig.add_trace(go.Scatter(x=df_rl.iloc[::100, :]['receive_ts'], y=df_rl.iloc[::100, :]['BTC'],
                         name='Inventory Size'), row=3, col=1)

fig.update_yaxes(title_text="USDT", row=1, col=1)
fig.update_yaxes(title_text="USDT", row=2, col=1)
fig.update_yaxes(title_text="BTC", row=3, col=1)
# fig.update_yaxes(title_text="Actions ID", row=4, col=1)

fig.update_layout(title_text="RL Strategy: maker fee = -0.004%", height=700)

fig.write_html('docs/RLStrategy_btc.html')
# fig.write_image('../images/results/RLStrategy.jpeg')
fig.show()

In [ ]:
fig = make_subplots(rows=3, cols=1)

fig.add_trace(go.Scatter(x=df_bp_fee.iloc[::100, :]['receive_ts'], y=df_bp_fee.iloc[::100, :]['BTC'],
                         name='Naive'), row=1, col=1)

fig.add_trace(go.Scatter(x=df_st_fee.iloc[::100, :]['receive_ts'], y=df_st_fee.iloc[::100, :]['BTC'],
                         name='Stoikov'), row=2, col=1)

fig.add_trace(go.Scatter(x=df_rl_fee.iloc[::100, :]['receive_ts'], y=df_rl_fee.iloc[::100, :]['BTC'],
                         name='RL'), row=3, col=1)

# fig.update_yaxes(title_text="USDT", row=1, col=1)
fig.update_yaxes(title_text="BTC", row=2, col=1)
# fig.update_yaxes(title_text="USDT", row=3, col=1)
# fig.update_yaxes(title_text="Actions ID", row=4, col=1)

fig.update_layout(title_text="Inventory", height=700)

# fig.write_html('../docs/RLStrategy_eth.html')
# fig.write_image('../images/results/RLStrategy.jpeg')
fig.show()

In [ ]:
fig = make_subplots(rows=1, cols=1)

fig.add_trace(go.Scatter(x=df_bp_fee.iloc[::100, :]['receive_ts'], y=df_bp_fee.iloc[::100, :]['mid_price'],
                         ), row=1, col=1)

# fig.add_trace(go.Scatter(x=df_st_fee.iloc[::100, :]['receive_ts'], y=df_st_fee.iloc[::100, :]['total'],
#                          name='Stoikov'), row=2, col=1)

# fig.add_trace(go.Scatter(x=df_rl_fee.iloc[::100, :]['receive_ts'], y=df_rl_fee.iloc[::100, :]['total'],
#                          name='RL'), row=3, col=1)

# fig.update_yaxes(title_text="USDT", row=1, col=1)
fig.update_yaxes(title_text="USDT", row=1, col=1)
# fig.update_yaxes(title_text="USDT", row=3, col=1)
# fig.update_yaxes(title_text="Actions ID", row=4, col=1)

fig.update_layout(title_text='Price', height=370, width=970)

# fig.write_html('../docs/RLStrategy.html')
# fig.write_image('../images/results/RLStrategy.jpeg')
# fig.show()

fig.update_traces(line_color='purple')